In [1]:
import numpy as np
from westpy.qdet import QDETResult
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.mappers import JordanWignerMapper
import pyscf

 
 _    _ _____ _____ _____            
| |  | |  ___/  ___|_   _|           
| |  | | |__ \ `--.  | |_ __  _   _  
| |/\| |  __| `--. \ | | '_ \| | | | 
\  /\  / |___/\__/ / | | |_) | |_| | 
 \/  \/\____/\____/  \_/ .__/ \__, | 
                       | |     __/ | 
                       |_|    |___/  
 
WEST version     :  6.0.0
Today            :  2024-11-04 12:22:30.577841


In [2]:

effective_hamiltonian = QDETResult(filename='wfreq.json')

fcisolver = pyscf.fci.direct_spin1.FCISolver()
heff = effective_hamiltonian.heff

h1e = heff.h1e
eri = heff.eri
norb = heff.norb

print(h1e.shape)
print(eri.shape)

(6, 6)
(6, 6, 6, 6)


In [3]:
occ = effective_hamiltonian.occupation
nel = np.sum(occ)
# nelec = (int(round(nel)) // 2, int(round(nel)) // 2)
nelec = (1, 1)
nroots = 10

evs, evcs = fcisolver.kernel(
                h1e=h1e, eri=eri, norb=norb, nelec=nelec, nroots=nroots
            )

In [4]:
print("Ground state: \n{} \nExcited states: \n{} ".format(evs[0], evs[1:]))

Ground state: 
0.3646078004802986 
Excited states: 
[0.40570138 0.40978584 0.41236237 0.41629868 0.418289   0.41922208
 0.42330409 0.42494106 0.44252749] 


In [5]:
hamiltonian = ElectronicEnergy.from_raw_integrals(h1e, eri)

In [6]:
hamiltonian.constants

{}

In [7]:
from qiskit_nature.second_q.problems.electronic_structure_problem import ElectronicStructureProblem
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.properties import ParticleNumber

In [8]:
problem = ElectronicStructureProblem(hamiltonian) 

In [9]:
problem.num_spatial_orbitals = 6
problem.basis = ElectronicBasis.MO
problem.num_particles = (1, 1)

In [10]:
problem.properties.particle_number = ParticleNumber(num_spatial_orbitals=6)

In [11]:
problem.properties.particle_number.second_q_ops()

{'ParticleNumber': FermionicOp({'+_0 -_0': 1.0, '+_1 -_1': 1.0, '+_2 -_2': 1.0, '+_3 -_3': 1.0, '+_4 -_4': 1.0, '+_5 -_5': 1.0, '+_6 -_6': 1.0, '+_7 -_7': 1.0, '+_8 -_8': 1.0, '+_9 -_9': 1.0, '+_10 -_10': 1.0, '+_11 -_11': 1.0}, num_spin_orbitals=12, )}

In [12]:
problem.orbital_occupations_b

array([1., 0., 0., 0., 0., 0.])

In [13]:
fermionic_op = problem.second_q_ops()[0]

In [14]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(fermionic_op)

In [15]:
print(qubit_hamiltonian.to_list())

[('IIIIIIIIIIII', (0.9072181651089145+0j)), ('IIIIIIIIIIIZ', (-0.005805247774101101+0j)), ('IIIIIIIIIIYY', (0.0002578737053345831+0j)), ('IIIIIIIIIIXX', (0.0002578737053345831+0j)), ('IIIIIIIIIYZY', (0.0009001188683270902+0j)), ('IIIIIIIIIXZX', (0.0009001188683270902+0j)), ('IIIIIIIIYZZY', (0.0005658137421822455+0j)), ('IIIIIIIIXZZX', (0.0005658137421822455+0j)), ('IIIIIIIYZZZY', (0.0033652423869124154+0j)), ('IIIIIIIXZZZX', (0.0033652423869124154+0j)), ('IIIIIIYZZZZY', (-0.0005213064307647502+0j)), ('IIIIIIXZZZZX', (-0.0005213064307647502+0j)), ('IIIIIIIIIIZI', (-0.02859422972100895+0j)), ('IIIIIIIIIYYI', (-0.0016796049559928028+0j)), ('IIIIIIIIIXXI', (-0.0016796049559928028+0j)), ('IIIIIIIIYZYI', (0.0002927932898682862+0j)), ('IIIIIIIIXZXI', (0.0002927932898682862+0j)), ('IIIIIIIYZZYI', (-5.7984266459491644e-05+0j)), ('IIIIIIIXZZXI', (-5.7984266459491644e-05+0j)), ('IIIIIIYZZZYI', (-0.0010641274551139647+0j)), ('IIIIIIXZZZXI', (-0.0010641274551139647+0j)), ('IIIIIIIIIZII', (-0.027609

In [16]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Estimator as AerEstimator

# ansatz = UCCSD(
#     6,
#     (1, 1),
#     mapper,
#     initial_state=HartreeFock(
#         6,
#         (1, 1),
#         mapper,
#     ),
# )

ansatz_tl = TwoLocal(
    num_qubits=problem.num_spin_orbitals,
    rotation_blocks=["ry", "rz"],
    entanglement_blocks="cx",
    entanglement="full",
    reps=2,
)

ideal_estimator = AerEstimator(
    run_options={"shots": 1024},
    approximation=True,
)


In [17]:
def filter_criterion_custom(eigenstate, eigenvalue, aux_values):
    num_particles_aux = aux_values["ParticleNumber"][0]
    return (
        np.isclose(num_particles_aux, 2)
    )

In [22]:
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    print(eval_count, mean)

In [ ]:
from qiskit_algorithms.optimizers import SLSQP, COBYLA

vqe_solver = VQE(ideal_estimator, ansatz_tl, COBYLA(maxiter=1000), callback=store_intermediate_result)
vqe_solver.filter_criterion = filter_criterion_custom
# vqe_solver.initial_point = [0.0] * ansatz.num_parameter

In [19]:
ansatz_tl.num_parameters

72

In [20]:
ansatz_tl.decompose().depth()

39

In [24]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

In [ ]:
res = calc.solve(problem)
print(res)

In [26]:
res.eigenvalues

array([0.90441287])

In [27]:
from qiskit_algorithms import NumPyMinimumEigensolver

solver = NumPyMinimumEigensolver(filter_criterion = problem.get_default_filter_criterion() )

In [28]:
calc = GroundStateEigensolver(mapper, solver)

In [ ]:
res = calc.solve(problem)
print(res)

In [30]:
res.eigenvalues

array([0.3646078])